In [30]:
"""  
Copyright (c) 2019-present NAVER Corp.
MIT License
"""

# -*- coding: utf-8 -*-
import sys
import os
import time
import argparse

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable

from PIL import Image

import cv2
from skimage import io
import numpy as np
import craft_utils
import imgproc
import file_utils
import json
import zipfile

from craft import CRAFT

from collections import OrderedDict

In [31]:
model_path = "craft_mlt_25k.onnx"

In [34]:
import onnxruntime as ort
import numpy
import cv2

# Load the ONNX model into memory
sess = ort.InferenceSession(model_path)

image = cv2.imread("E:/Projects/DATA/license_plate/plates/18.jpg")
img_resized, target_ratio, size_heatmap = imgproc.resize_aspect_ratio(image, 1280, interpolation=cv2.INTER_LINEAR, mag_ratio=1.5)
ratio_h = ratio_w = 1 / target_ratio

# preprocessing
x = imgproc.normalizeMeanVariance(img_resized)
x = torch.from_numpy(x).permute(2, 0, 1)    # [h, w, c] to [c, h, w]
x = Variable(x.unsqueeze(0))                # [c, h, w] to [b, c, h, w]

# Prepare the input data
input_data = {
    "input": x
}

# Run the model
output_data = sess.run(None, input_data)

# Postprocess the output data
print(output_data)


tensor([[[[ 1.1187,  1.1015,  1.0844,  ..., -2.1179, -2.1179, -2.1179],
          [ 1.1015,  1.1015,  1.1015,  ..., -2.1179, -2.1179, -2.1179],
          [ 1.1015,  1.1015,  1.1015,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

         [[ 1.1331,  1.1155,  1.0980,  ..., -2.0357, -2.0357, -2.0357],
          [ 1.1155,  1.1155,  1.0980,  ..., -2.0357, -2.0357, -2.0357],
          [ 1.1155,  1.1155,  1.1155,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

         [[ 1.9951,  1.9777,  1.9603,  ..., -1.8044, -1.8044, -1.8044],
          [ 1.9777,  1.9777,  

In [35]:
import onnx

# Load the ONNX model into memory
model = onnx.load(model_path)

# Print the shape of the input tensor
input_shape = model.graph.input[0].type.tensor_type.shape.dim
print("Input shape:", [dim.dim_value for dim in input_shape])

Input shape: [1, 1, 32, 100]
